## Import Accident Data

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("train.csv", parse_dates = ['Occurrence Local Date Time'])
data.head()

,EventId,Occurrence Local Date Time,Reporting Agency,Cause,Subcause,Status,longitude,latitude,road_segment_id
0,60558,2016-01-01 00:53:00,Cam,Stationary Vehicle,Vehicle On Shoulder,Closed,18.5408955,-33.888275,S0B3CGQ
1,60559,2016-01-01 00:54:00,CAMERA,Accident,With A Fixed Object,Closed,18.93075632,-34.140857,RYJYAPI
2,60560,2016-01-01 02:26:00,Law Enforcement,Accident,Multi Vehicle,Closed,18.5533575,-33.959154,U3KP57C
3,60561,2016-01-01 02:56:00,CAMERA,Stationary Vehicle,Vehicle On Shoulder,Closed,18.67755616,-33.895258,RY0TRQ8
4,60562,2016-01-01 03:40:00,CAMERA,Accident,Multi Vehicle,Closed,18.83713197,-34.087051,8LOVJZ3


In [3]:
data.drop(["Reporting Agency", "Cause", "Subcause", "Status"], inplace=True, axis =1)

In [4]:
data.drop(data[data['longitude'] == 'Closed'].index, inplace=True)

In [5]:
data.shape

(53840, 5)

## Create Training DF for Each Hour/Road Segment

In [6]:
train = data

In [7]:
# Create a dataframe with a column for each segment_id (sid)
# Each row represents an hour.

sids = data['road_segment_id'].unique()

dts = pd.date_range('2016-01-01',
                    '2018-12-31',
                    freq="1h")
tr = pd.DataFrame({'datetime':dts})

for sid in sids:
    tr[str(sid)] = 0
    events = train.loc[train['road_segment_id'] == sid]
    dts = events['Occurrence Local Date Time'].dt.round('H')
    dates = dts.astype(str).unique()
    tr.loc[tr['datetime'].isin(dates), sid] = 1
tr.head()

,datetime,S0B3CGQ,RYJYAPI,U3KP57C,RY0TRQ8,8LOVJZ3,X4UA382,0QR8FDW,DZABHQW,EKZN1VM,...,YVR8GT6,ZAVM3PJ,DS4NLQE,HR19LL7,1451FOG,2ON8NSO,NFUEAN5,4T821GV,J6A19TW,43RCYZH
0,2016-01-01 00:00:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016-01-01 01:00:00,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016-01-01 02:00:00,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2016-01-01 03:00:00,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2016-01-01 04:00:00,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
tr.shape

(26281, 545)

In [11]:
# Reshape this as in sample submission
train = pd.DataFrame({
    'datetime x segment_id':np.concatenate([[str(x) + " x " + str(c) 
                                             for c in sids] 
                                            for x in tr['datetime']]),
    'datetime':np.concatenate([[str(x) for c in sids] for x in tr['datetime']]),
    'segment_id':np.concatenate([[str(c) for c in sids] for x in tr['datetime']]),
    'y':tr[sids].values.flatten()
})
train.head()

,datetime x segment_id,datetime,segment_id,y
0,2016-01-01 00:00:00 x S0B3CGQ,2016-01-01 00:00:00,S0B3CGQ,0
1,2016-01-01 00:00:00 x RYJYAPI,2016-01-01 00:00:00,RYJYAPI,0
2,2016-01-01 00:00:00 x U3KP57C,2016-01-01 00:00:00,U3KP57C,0
3,2016-01-01 00:00:00 x RY0TRQ8,2016-01-01 00:00:00,RY0TRQ8,0
4,2016-01-01 00:00:00 x 8LOVJZ3,2016-01-01 00:00:00,8LOVJZ3,0


In [12]:
train['datetime2'] = train['datetime'] 
train['datetime'] = pd.to_datetime(train['datetime'])
train['min'] = train['datetime'].dt.hour*60+train['datetime'].dt.minute

## Import Holiday Data

In [13]:
holiday = pd.read_csv("holiday.csv")

In [14]:
holiday['Date'] = pd.to_datetime(holiday['Date'])

## Merge Training and Holiday Data

In [15]:
train_holiday = pd.merge(train, holiday, left_on='datetime', right_on='Date', how = "left")
train_holiday.head()

,datetime x segment_id,datetime,segment_id,y,datetime2,min,Date,Day,Name
0,2016-01-01 00:00:00 x S0B3CGQ,2016-01-01,S0B3CGQ,0,2016-01-01 00:00:00,0,2016-01-01,Friday,New Year's Day
1,2016-01-01 00:00:00 x RYJYAPI,2016-01-01,RYJYAPI,0,2016-01-01 00:00:00,0,2016-01-01,Friday,New Year's Day
2,2016-01-01 00:00:00 x U3KP57C,2016-01-01,U3KP57C,0,2016-01-01 00:00:00,0,2016-01-01,Friday,New Year's Day
3,2016-01-01 00:00:00 x RY0TRQ8,2016-01-01,RY0TRQ8,0,2016-01-01 00:00:00,0,2016-01-01,Friday,New Year's Day
4,2016-01-01 00:00:00 x 8LOVJZ3,2016-01-01,8LOVJZ3,0,2016-01-01 00:00:00,0,2016-01-01,Friday,New Year's Day


In [16]:
train_holiday.drop(['Date', 'Day'], inplace=True, axis=1)

### Holiday Column Changed to 1s and 0s

In [17]:
train_holiday["holiday"] = np.where(train_holiday['Name'].notna(), 1, 0)

In [18]:
train_holiday.drop('Name', inplace=True, axis=1)

## Import Road Segment Data

In [19]:
road = pd.read_csv('road_segments.csv')


In [20]:
road.drop(["REGION", "SURFTYPE", "PAVETYPE", "length_1"], inplace=True, axis = 1)


## Merge Training and Road Data

In [21]:
data, train, updated_new, tr = 0, 0, 0, 0

In [23]:
merged = pd.merge(train_holiday, road, on = 'segment_id', how="left")


### Change Categorical Variables to Dummies

In [24]:
dummiesw = pd.get_dummies(merged.WIDTH) #one hot encoding for width
dummiesl = pd.get_dummies(merged.LANES)  #one hot encoding for lanes
dummiesc = pd.get_dummies(merged.CLASS)  #one hot encoding for class

In [25]:
dummiesco = pd.get_dummies(merged.CONDITION) #one hot encoding for condition
dummiesr = pd.get_dummies(merged.ROADNO) #one hot encoding for road number

In [26]:
merged = pd.concat([merged, dummiesw, dummiesl, dummiesc, dummiesco, dummiesr], axis = 1)

In [27]:
final = merged.drop(["WIDTH", "CLASS", "LANES", "CONDITION", "ROADNO"], axis = 1)

In [28]:
dummiesw, dummiesl, dummiesc, dummiesco, dummiesr = 0, 0, 0, 0, 0

In [29]:
final= final.rename(columns = {
    "day": "Day-of-Week",
    "X": "Latitude",
    "Y": "Longitude",
    "datetime_x": "datetime"
})

## Merge Training and Weather

In [33]:
weather_airport = pd.read_csv("weather_cleaned.csv")
weather_airport.head()

,Unnamed: 0,datetime,T,Po,P,Pa,U,Ff,ff10,VV,Td
0,0,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
1,1,2016-01-01 03:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
2,2,2016-01-01 04:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
3,3,2016-01-01 05:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
4,4,2016-01-01 06:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6


In [34]:
weather_airport.drop(['Unnamed: 0'], inplace=True, axis=1)

In [35]:
final_updated = final.loc[final['datetime2'] >= '2016-01-01 02:00:00']

In [36]:
final_updated.datetime2 = final_updated.datetime2.astype(str)
weather_airport.datetime = weather_airport.datetime.astype(str)

C:\Users\steph\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [37]:
final2 = pd.merge(final_updated, weather_airport, left_on = 'datetime2', right_on = "datetime", how="left")
final2.head()

,datetime x segment_id,datetime_x,segment_id,y,datetime2,min,holiday,Latitude,Longitude,0.0,...,datetime_y,T,Po,P,Pa,U,Ff,ff10,VV,Td
0,2016-01-01 02:00:00 x S0B3CGQ,2016-01-01 02:00:00,S0B3CGQ,0,2016-01-01 02:00:00,120,0,18.541817,-33.889284,1,...,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
1,2016-01-01 02:00:00 x RYJYAPI,2016-01-01 02:00:00,RYJYAPI,0,2016-01-01 02:00:00,120,0,18.930836,-34.140648,0,...,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
2,2016-01-01 02:00:00 x U3KP57C,2016-01-01 02:00:00,U3KP57C,1,2016-01-01 02:00:00,120,0,18.551477,-33.958777,1,...,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
3,2016-01-01 02:00:00 x RY0TRQ8,2016-01-01 02:00:00,RY0TRQ8,0,2016-01-01 02:00:00,120,0,18.677457,-33.897177,0,...,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6
4,2016-01-01 02:00:00 x 8LOVJZ3,2016-01-01 02:00:00,8LOVJZ3,0,2016-01-01 02:00:00,120,0,18.835946,-34.085912,0,...,2016-01-01 02:00:00,20.9,756.3,760.3,-0.003512,760.3,4.0,8.0,30.0,18.6


In [38]:
final2['day'] = final2['datetime_x'].dt.dayofweek

In [39]:
dummiesd = pd.get_dummies(final2.day) 

In [40]:
final3 = pd.concat([final2, dummiesd], axis = 1)

In [41]:
final, final2, final_updated, weather_airport, merged = 0, 0, 0, 0, 0

In [42]:
final3.y.value_counts(normalize=True)

0    0.996405
1    0.003595
Name: y, dtype: float64

In [43]:
final3.drop(["VV"], inplace=True, axis=1)

In [44]:
numerical_train = final3.select_dtypes(np.number)

In [45]:
X_train = numerical_train.drop(["y"], axis = 1)

In [46]:
y_train = final3["y"]

In [47]:
dummiesd = 0
dummiesy = 0
dummiesw = 0
dummiesl = 0
dummiesco = 0
dummiesr = 0 
dummiesc = 0

In [48]:
X_train.columns

Index([         'min',      'holiday',     'Latitude',    'Longitude',
                  0.0,            7.4,           12.8,           20.2,
                    0,              2,      'Primary',     'Tertiary',
               'Fair',         'Good',      'Unknown',    'Very Good',
                'M17',           'M3',           'M4',           'M5',
                 'N1',           'N2',           'N7',         'R300',
       'Unclassified',            'T',           'Po',            'P',
                 'Pa',            'U',           'Ff',         'ff10',
                 'Td',          'day',              0,              1,
                    2,              3,              4,              5,
                    6],
      dtype='object')

## Random Forest

In [50]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [51]:
model = BalancedRandomForestClassifier(n_estimators=10, class_weight= "balanced", max_features = "log2")

In [52]:
model.fit(X_train, y_train)

BalancedRandomForestClassifier(class_weight='balanced', max_features='log2',
                               n_estimators=10)

## 2019 Data to Make Prediction

In [53]:
dts = pd.date_range('2019-01-01 00:00:00',
                    '2019-03-31 23:00:00',
                    freq="1h")
tr = pd.DataFrame({'datetime':dts})

for sid in sids:
    tr[str(sid)] = 0
    
ss = pd.DataFrame({
    'datetime x segment_id':np.concatenate([[str(x) + " x " + str(c)  
                                            for x in tr['datetime']for c in sids]]),
    'datetime':np.concatenate([[str(x) for x in tr['datetime']for c in sids]]),
    'segment_id':np.concatenate([[str(c) for x in tr['datetime']for c in sids]])
})
ss.head()

,datetime x segment_id,datetime,segment_id
0,2019-01-01 00:00:00 x S0B3CGQ,2019-01-01 00:00:00,S0B3CGQ
1,2019-01-01 00:00:00 x RYJYAPI,2019-01-01 00:00:00,RYJYAPI
2,2019-01-01 00:00:00 x U3KP57C,2019-01-01 00:00:00,U3KP57C
3,2019-01-01 00:00:00 x RY0TRQ8,2019-01-01 00:00:00,RY0TRQ8
4,2019-01-01 00:00:00 x 8LOVJZ3,2019-01-01 00:00:00,8LOVJZ3


In [54]:
# Add the extra features
ss['datetime'] = pd.to_datetime(ss['datetime'])
ss['date'] = ss["datetime"].dt.date
ss['min'] = ss['datetime'].dt.hour*60+ss['datetime'].dt.minute
ss.head()

,datetime x segment_id,datetime,segment_id,date,min
0,2019-01-01 00:00:00 x S0B3CGQ,2019-01-01,S0B3CGQ,2019-01-01,0
1,2019-01-01 00:00:00 x RYJYAPI,2019-01-01,RYJYAPI,2019-01-01,0
2,2019-01-01 00:00:00 x U3KP57C,2019-01-01,U3KP57C,2019-01-01,0
3,2019-01-01 00:00:00 x RY0TRQ8,2019-01-01,RY0TRQ8,2019-01-01,0
4,2019-01-01 00:00:00 x 8LOVJZ3,2019-01-01,8LOVJZ3,2019-01-01,0


In [55]:
ss.date = pd.to_datetime(ss['date'])

In [56]:
ss_holiday = pd.merge(ss, holiday, left_on='date', right_on='Date', how = "left")

In [57]:
ss_holiday.drop(['Date', 'Day'], inplace=True, axis=1)

In [58]:
ss_holiday["holiday"] = np.where(ss_holiday['Name'].notna(), 1, 0)

In [59]:
ss_holiday["holiday"].value_counts()

0    1148928
1      26112
Name: holiday, dtype: int64

In [60]:
ss_holiday.drop('Name', inplace=True, axis=1)

In [61]:
ss_road = pd.merge(ss_holiday, road, on = 'segment_id', how="left")

In [62]:
dummiesw = pd.get_dummies(ss_road.WIDTH) #one hot encoding for width
dummiesl = pd.get_dummies(ss_road.LANES)  #one hot encoding for lanes
dummiesc = pd.get_dummies(ss_road.CLASS)  #one hot encoding for class
dummiesco = pd.get_dummies(ss_road.CONDITION) #one hot encoding for condition
dummiesr = pd.get_dummies(ss_road.ROADNO) #one hot encoding for road number

In [63]:
merged2 = pd.concat([ss_road, dummiesw, dummiesl, dummiesc, dummiesco, dummiesr], axis = 1)

In [64]:
ss_final = merged2.drop(["WIDTH", "CLASS", "LANES", "CONDITION", "ROADNO"], axis = 1)

In [65]:
ss_final= ss_final.rename(columns = {
    "day": "Day-of-Week",
    "X": "Latitude",
    "Y": "Longitude",
    "datetime_x": "datetime"
})

In [67]:
weather_2019 = pd.read_csv("weather_2019.csv")
weather_2019.head()

,Unnamed: 0,Local time in Cape Town,T,Po,P,Pa,U,DD,Ff,ff10,Td
0,0,2019-01-01 02:00:00,19.2,756.7,760.9,-0.6,760.9,Wind blowing from the south-southwest,5.0,7.000000,16.5
1,1,2019-01-01 03:00:00,19.2,756.7,760.9,-0.6,760.9,Wind blowing from the south-southwest,5.0,7.000000,16.5
2,2,2019-01-01 04:00:00,19.2,756.7,760.9,-0.6,760.9,Wind blowing from the south-southwest,5.0,7.000000,16.5
3,3,2019-01-01 05:00:00,19.1,757.3,761.3,0.6,761.3,Wind blowing from the south-southwest,4.0,6.926041,14.4
4,4,2019-01-01 06:00:00,19.1,757.3,761.3,0.6,761.3,Wind blowing from the south-southwest,4.0,6.926041,14.4


In [68]:
weather_2019.drop(["Unnamed: 0", "DD"], inplace=True, axis=1)

In [69]:
weather_2019['Local time in Cape Town'] = weather_2019['Local time in Cape Town'].astype('O')

In [70]:
ss_final['datetime'] = ss_final['datetime'].astype('O')

In [71]:
ss_final.datetime = ss_final.datetime.astype(str)
weather_2019['Local time in Cape Town'] = weather_2019['Local time in Cape Town'].astype(str)

In [72]:
ss_final2 = pd.merge(ss_final, weather_2019, left_on = 'datetime', right_on = "Local time in Cape Town", how="left")
ss_final2.head()

,datetime x segment_id,datetime,segment_id,date,min,holiday,Latitude,Longitude,0.0,7.4,...,Unclassified,Local time in Cape Town,T,Po,P,Pa,U,Ff,ff10,Td
0,2019-01-01 00:00:00 x S0B3CGQ,2019-01-01 00:00:00,S0B3CGQ,2019-01-01,0,1,18.541817,-33.889284,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 00:00:00 x RYJYAPI,2019-01-01 00:00:00,RYJYAPI,2019-01-01,0,1,18.930836,-34.140648,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 00:00:00 x U3KP57C,2019-01-01 00:00:00,U3KP57C,2019-01-01,0,1,18.551477,-33.958777,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 00:00:00 x RY0TRQ8,2019-01-01 00:00:00,RY0TRQ8,2019-01-01,0,1,18.677457,-33.897177,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 00:00:00 x 8LOVJZ3,2019-01-01 00:00:00,8LOVJZ3,2019-01-01,0,1,18.835946,-34.085912,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
ss_final2 = ss_final2.bfill()

In [74]:
ss_final2["datetime2"] = pd.to_datetime(ss_final2["datetime"])

In [75]:
ss_final2['day'] = ss_final2['datetime2'].dt.dayofweek

In [76]:
dummiesd = pd.get_dummies(ss_final2.day) 

In [77]:
ss_final3 = pd.concat([ss_final2, dummiesd], axis = 1)

In [78]:
ss_final3.drop(['date', 'Local time in Cape Town'], inplace=True, axis=1)

## Use Model to Make Prediction

In [79]:
numerical = ss_final3.select_dtypes(np.number)

In [80]:
ss_final3['prediction'] = model.predict(numerical)
ss_final3.head()

,datetime x segment_id,datetime,segment_id,min,holiday,Latitude,Longitude,0.0,7.4,12.8,...,datetime2,day,0,1,2,3,4,5,6,prediction
0,2019-01-01 00:00:00 x S0B3CGQ,2019-01-01 00:00:00,S0B3CGQ,0,1,18.541817,-33.889284,1,0,0,...,2019-01-01,1,0,1,0,0,0,0,0,1
1,2019-01-01 00:00:00 x RYJYAPI,2019-01-01 00:00:00,RYJYAPI,0,1,18.930836,-34.140648,0,0,1,...,2019-01-01,1,0,1,0,0,0,0,0,0
2,2019-01-01 00:00:00 x U3KP57C,2019-01-01 00:00:00,U3KP57C,0,1,18.551477,-33.958777,1,0,0,...,2019-01-01,1,0,1,0,0,0,0,0,0
3,2019-01-01 00:00:00 x RY0TRQ8,2019-01-01 00:00:00,RY0TRQ8,0,1,18.677457,-33.897177,0,0,0,...,2019-01-01,1,0,1,0,0,0,0,0,0
4,2019-01-01 00:00:00 x 8LOVJZ3,2019-01-01 00:00:00,8LOVJZ3,0,1,18.835946,-34.085912,0,0,0,...,2019-01-01,1,0,1,0,0,0,0,0,1


In [81]:
ss_final3['prediction'].sum()

253519

In [82]:
ss_final3.shape

(1175040, 46)

In [80]:
# Save to CSV and submit
ss_final3[['datetime x segment_id', 'prediction']].to_csv('submission5.csv', index=False)